# Hartree-Fock

The Hartree-Fock method is an uncorrelated mean-field theory that offers a qualitative description of chemical systems. Although Hartree-Fock theory is only qualitatively correct, it forms the basis for more accurate models and becomes the cornerstone of ab initio quantum chemistry.

Hartree-Fock method assumes that the ground state wave function $\vert\Phi_0\rangle$ is an antisymmetrized product state with a single-electron orbital basis $\{\phi_i\}$. It is often represented by a single Slater determinant:

\begin{equation}
\Phi_0(x_1,x_2,\dots,x_N) = \frac{1}{(\sqrt{N!})}
\begin{vmatrix}
\phi_1(x_1) & \phi_2(x_1) & \dots & \phi_N(x_1) \\
\phi_1(x_2) & \phi_2(x_2) & \dots & \phi_N(x_2) \\
\vdots      & \vdots      & \dots & \vdots \\
\phi_1(x_N) & \phi_2(x_N) & \dots & \phi_N(x_N) \\
\end{vmatrix}
\end{equation}

To optimize the Hartree-Fock wave function, we invoke the variation principle to minimize the expectation value of the energy:

\begin{equation}
E_{HF} = \mathop{min}\limits_{\{\phi_i\}}^{} \langle\Phi_0\vert\hat{H}\vert\Phi_0\rangle
\end{equation}

The minimization of the expected energy with a given basis set leads to the Roothan-Hartree-Fock equation:

\begin{equation}
\mathbf{F(C)C} = \mathbf{SC\epsilon}
\end{equation}

where $\mathbf{S}$ is the overlap matrix with atomic basis set, $\mathbf{C}$ is the matrix of molecular orbital coefficients, $\mathbf{\epsilon}$ is a diagonal matrix whose elements represent the eigenenergies, and the $\mathbf{F}$ is the Fock matrix, which depends on the coefficients matrix $\mathbf{C}$.

The Fock matrix $\mathbf{F}$ is:

\begin{equation}
\mathbf{F(C)} = \mathbf{H} + \mathbf{G(C)}
\end{equation}

$\mathbf{H}$ is the core-Hamiltonian matrix, and it is the sum of kinetic energy integral $\mathbf{T}$ and electron-nuclear attraction integral $\mathbf{V}$. Both are one-electron integral and invariance during the self-consistent field optimization.

\begin{equation}
\mathbf{H} = \mathbf{T} + \mathbf{V}
\end{equation}

$\mathbf{G}$ is Hartree-Fock potential matrix, and it is the sum of coulomb matrix $\mathbf{J}$ and the exchange matrix $\mathbf{K}$. Both of them are two-electron integral matrix and is a function of $\mathbf{C}$.

\begin{equation}
\mathbf{G(C)} = \mathbf{J(C)} + \mathbf{K(C)}
\end{equation}

Hartree-Fock energy is an upper bound to the exact energy using the variational theorem. It typically gives errors of $0.5\%$ in the energy, $1\%$ in molecular geometry, and $5−10\%$ in other properties. The correlation energy is the difference between the Hartree-Fock limit energy and a system's exact non-relativistic energy.

\begin{equation}
\begin{aligned}
E_{HF} &\geq E_{exact}\\
E_{corr} &= E_{exact} - E_{HF}
\end{aligned}
\end{equation}

## Plain Solver


Since the Hartree-Fock-Roothaan equations are nonlinear, we will solve them with an iterative procedure. Such a procedure is often referred to as a self-consistent field (SCF) algorithm, and here we introduce the most naive one.

In [1]:
from moha import *
import numpy as np
from moha.basis import BasisSet
from moha.molecule import Molecule
from moha.hamiltonian import Hamiltonian

np.set_printoptions(precision=5,suppress=True)

geo = [[8,  0.000000000000,  -0.143225816552,  0.000000000000],
    ['h',   1.638036840407,   1.136548822547,  -0.000000000000],
    ["H",  -1.638036840407,   1.136548822547,  -0.000000000000]]

mol = Molecule.build(geo,pg=False)
orb = BasisSet.build(mol,'sto-3g.nwchem')

ham = Hamiltonian.build(mol,orb)

sym = Symmetry(n=10,ms2=0,ipg=0)

hf_solver = HFSolver(ham,sym)
hf_energy,hf_wfn = hf_solver.rhf()

print(hf_energy+mol.e_nuc)

-74.94207993214405


In [2]:
from moha.property.multipole_moment import Moment

mm_results = Moment(mol,orb,hf_wfn).dipole()
print(mm_results)

[[ 0.       0.       0.05079  0.       0.       0.00223 -0.00223]
 [ 0.       0.       0.64117  0.       0.       0.26274 -0.26274]
 [ 0.05079  0.64117  0.       0.       0.       0.43763  0.43763]
 [ 0.       0.       0.       0.       0.       0.1474  -0.1474 ]
 [ 0.       0.       0.       0.       0.       0.       0.     ]
 [ 0.00223  0.26274  0.43763  0.1474   0.       1.63804 -0.     ]
 [-0.00223 -0.26274  0.43763 -0.1474   0.      -0.      -1.63804]]
[[-0.14323 -0.0339   0.       0.05079  0.      -0.00376 -0.00376]
 [-0.0339  -0.14323  0.       0.64117  0.       0.14997  0.14997]
 [ 0.       0.      -0.14323  0.       0.       0.10895 -0.10895]
 [ 0.05079  0.64117  0.       0.       0.       0.33409  0.33409]
 [ 0.       0.       0.       0.      -0.14323  0.       0.     ]
 [-0.00376  0.14997  0.10895  0.33409  0.       1.13655  0.20658]
 [-0.00376  0.14997 -0.10895  0.33409  0.       0.20658  1.13655]]
[[0.      0.      0.      0.      0.05079 0.      0.     ]
 [0.      0.   